In [13]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import uuid


import urllib.parse as urlparse
from urllib.parse import parse_qs
from urllib.parse import urlparse
from urllib.request import urlopen
import time
import re

In [14]:
my_header = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
              + "(KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36"}

querystring = {"page":"20", 'product_url':'https://www.walmart.com/search/?query=watch'}
item_url=[]
prod_name=[]
total_pages=int(querystring["page"])


for i in range(1,total_pages+1):
    
    url='https://www.walmart.com/search/?page='+ str(i) +'&ps=48&query=watches'
    r= requests.get(url, headers= my_header)
    price_per_page=[]
    if r.status_code==200:
        soup_main= BeautifulSoup(r.content, 'html')
        #print(soup_main)
        summary=soup_main.find('div', {'class':'search-product-result', 'id':'searchProductResult'})
        product_list= summary.find_all('li')
        for prod in product_list:
            try:
                
                item_url.append(prod.find('a', {"class":"product-title-link line-clamp line-clamp-2 truncate-title"}).get('href'))
            except:
                pass
    else:
        print("Error-",r.status_code)

In [15]:
product_code=[u.split('/')[-1] for u in item_url]

In [31]:
upc=[]
item_ID=[]
product_name=[]
for prod_code in product_code:
    item_url= 'https://www.walmart.com/reviews/product/'+ prod_code +'?page=2'
    r = requests.get(item_url,headers=proxy_headers)

    r.status_code

    soup = BeautifulSoup(urlopen(item_url),'html.parser')
    for val in soup.find_all("script"):
        #print(val)
        if 'upc' in str(val):
            val=str(val)
            prob_dict = val.split('upc')[1]
            UPC=prob_dict.split(',')[0]
            UPC=UPC[3:-1]
            upc.append(UPC)
            prob_dict = val.split('usItemId')[1]
            item_id=prob_dict.split(',')[0]
            item_id=item_id[3:-1]
            item_ID.append(item_id)
            prob_dict = val.split('productName')[1]
            product=prob_dict.split(',')[0]
            product=product[3:-1]
            product_name.append(product)
            


    

In [37]:
df_dict={"Product Name": product_name, "Walmart Product Code": item_ID, "UPC": upc}

In [38]:
df=pd.DataFrame(df_dict)
df.head()

,Product Name,Walmart Product Code,UPC
0,Casio Men's Classic Resin Strap Sport Watch F9...,20670078,079767246280
1,DZ09 Bluetooth Smart Wrist Watch With Health M...,939405652,701649348941
2,Classic Analog Water Resistant Watch w/ Resin ...,45050246,079767234072
3,iTouch Sport Smartwatch Fitness Tracker Heart ...,173019570,846692617504
4,Casio Men's Classic Digital Illuminator Watch ...,894302,079767541255


In [40]:
df.to_csv("UPCs_of_Walmart_Watches.csv")